In [ ]:
### COMPARING MEANS ###

In [ ]:
from math import sqrt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pydataset import data
import statistics
def get_db_url(db):
    from env import username, password, host
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'
url = get_db_url("telco_churn")
url2 = get_db_url("employees")
alpha = 0.05

In [ ]:
# Answer with the type of test you would use (assume normal distribution):

In [ ]:
# Is there a difference in grades of students on the second floor compared to grades of all students?

In [ ]:
# One sample T-Test

In [ ]:
# Are adults who drink milk taller than adults who dont drink milk?

In [ ]:
# Independent T-Test

In [ ]:
# Is the the price of gas higher in texas or in new mexico?

In [ ]:
# Independent T-Test

In [ ]:
# Are there differences in stress levels between students who take data science vs students who take web development vs students who take cloud academy?

In [ ]:
# ANOVA Test

In [ ]:
# Ace Realty wants to determine whether the average time it takes to sell homes is different for its two offices.
# A sample of 40 sales from office #1 revealed a mean of 90 days and a standard deviation of 15 days.
# A sample of 50 sales from office #2 revealed a mean of 100 days and a standard deviation of 20 days.
# Use a .05 level of significance.

In [ ]:
office_1 = stats.norm(90, 15).rvs(40)
office_2 = stats.norm(100, 20).rvs(50)

In [ ]:
stat, p_val = stats.levene(office_1, office_2)

if p_val < 0.05:
    print('We can reject H0 ==> inequal variance')
    equal_var = False
else:
    print('We fail to reject H0 ==> equal variance')
    equal_var = True

In [ ]:
α = 0.05

t_stat, p_val = stats.ttest_ind(office_1, office_2, equal_var=equal_var)
print(f't-stat: {t_stat}')
print(f'p-value: {p_val}')

In [ ]:
print(f'''
p-value ({p_val}) is less than alpha value ({α}), we reject the null hypothesis''')

In [ ]:
# Load the mpg dataset and use it to answer the following questions:

In [ ]:
mpg = data('mpg')

In [ ]:
# Is there a difference in fuel-efficiency in cars from 2008 vs 1999?

In [ ]:
mpg['avg_fe'] = stats.hmean(mpg[['cty', 'hwy']], axis =1)

fe_2008 = mpg[mpg.year == 2008].avg_fe
fe_1999 = mpg[mpg.year == 1999].avg_fe
stats.levene(fe_2008, fe_1999)

t, p = stats.ttest_ind(fe_2008, fe_1999)
t, p

In [ ]:
# Are compact cars more fuel-efficient than the average car?

In [ ]:
fe_compact = mpg[mpg['class'] == 'compact'].avg_fe
μ = mpg.avg_fe.mean()

In [ ]:
t, p = stats.ttest_1samp(fe_compact, μ)
t, p/2, alpha

In [ ]:
print(f'''p/2 ({p/2:.11f}) < alpha (.05) and t > 0, we reject the null hypothesis.''')

In [ ]:
# Do manual cars get better gas mileage than automatic cars?

In [ ]:
fe_auto = mpg[mpg.trans.str.contains('auto')].avg_fe
fe_manual = mpg[mpg.trans.str.contains('manual')].avg_fe

In [ ]:
stats.levene(fe_auto, fe_manual)

In [ ]:
t, p = stats.ttest_ind(fe_manual, fe_auto, equal_var = True)
t, p/2, alpha

In [ ]:
### CORRELATION ###

In [ ]:
# Is there a relationship between the length of your arm and the length of your foot?

In [ ]:
# Independent T test

In [ ]:
# Do guys and gals quit their jobs at the same rate?

In [ ]:
# Independent T test

In [ ]:
# Does the length of time of the lecture correlate with a students grade?

In [ ]:
# Sample T Test

In [ ]:
# Use the telco_churn data

In [ ]:
# Does tenure correlate with monthly charges?

In [ ]:
confidence_level = 0.95
alpha = 1 - confidence_level
alpha

In [ ]:
sql = """
select *
from telco_churn """
df = pd.read_sql(sql, url)

In [ ]:
corr, p = stats.pearsonr(df.tenure_month, df.monthly_charges)

if p < alpha:
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")
    
corr, p

In [ ]:
# Total charges?

In [ ]:
corr, p = stats.pearsonr(df.tenure_month, df.total_charges)

if p < alpha:
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")
    
corr, p

In [ ]:
# What happens if you control for phone and internet service?

In [ ]:
df["telecom_setup"] = df.internet_type + " " + df.phone_type
options = df.telecom_setup.value_counts().index

output = []

for option in options:
    subset = df[df.telecom_setup == option]
    corr, p, message = correlation_test(subset.tenure_month, subset.monthly_charges)
    result = {
        "option": option,
        "correlation": corr,
        "p": p,
        "message": message
    }
    output.append(result)

print("Correlation test for tenure vs. monthly_charges, controling for Internet/Phone service")
pd.DataFrame(output)

In [ ]:
# Is there a relationship between how long an employee has been with the company and their salary?

In [ ]:
sql2 = """
select datediff(curdate(), hire_date) as days_on_job, salary
from employees
join salaries using(emp_no)
where to_date > curdate();
"""

In [ ]:
df2 = pd.read_sql(sql2, url2)
df2

In [ ]:
alpha = 0.05
corr, p = stats.pearsonr(df2.days_on_job, df2.salary)

if p < alpha:
    print("reject null hypothesis")
    print("Evidence supports that there is a linear correlation")
else:
    print("Failed reject the null hypothesis")

corr, p

In [ ]:
# Is there a relationship between how long an employee has been with the company and the number of titles they have had?

In [ ]:
sql3 = """
select emp_no, count(title) as title_count, datediff(curdate(), hire_date) as days_on_job
from employees
join titles using(emp_no)
group by emp_no;
"""
df3 = pd.read_sql(sql3, url2)
df3.head()

In [ ]:
corr, p = stats.pearsonr(df3.days_on_job, df3.title_count)

if p < alpha:
    print("Reject null hypothesis")
else:
    print("Failed reject the null hypothesis")

corr, p

In [ ]:
# Is there a relationship between days and reaction time?

In [ ]:
from pydataset import data
df4 = data("sleepstudy")

In [ ]:
corr, p = stats.pearsonr(df4.Days, df4.Reaction)

if p < alpha:
    print("Reject null hypothesis")
else:
    print("Failed to reject the null hypothesis")

corr, p

In [ ]:
### COMPARISON OF GROUPS ###

In [ ]:
# Do students get better test grades if they have a rubber duck on their desk?

In [ ]:
# Sample T Test

In [ ]:
# Does smoking affect when or not someone has lung cancer?

In [ ]:
# Independent T Test

In [ ]:
# Is gender independent of a person’s blood type?

In [ ]:
# Independent T Test

In [ ]:
# A farming company wants to know if a new fertilizer has improved crop yield or not

In [ ]:
# Sample T Test

In [ ]:
# Does the length of time of the lecture correlate with a students grade?

In [ ]:
# Sample T Test

In [ ]:
# Do people with dogs live in apartments more than people with cats?

In [ ]:
# ANOVA

In [ ]:
# help answer the question of whether using a macbook and being a codeup student are independent of each othe

In [ ]:
index = ["Uses Macbook", "Doesn't Macbook"]
columns = ["Codeup Student", "Not Codeup Student"]

obv = pd.DataFrame([[49, 20], [1, 30]], index=index, columns=columns)

In [ ]:
chi2, p, degf, expected = stats.chi2_contingency(obv)
p

In [ ]:
# Choose another 2 categorical variables from the mpg dataset and perform a 
# chi2 contingency table test with them. Be sure to state your null and alternative hypotheses.

In [ ]:
df = data("mpg")
df = df.rename(columns={"class": "vehicle_type"})
df["is_automatic"] = df.trans.str.startswith("a")

In [ ]:
obv = pd.crosstab(df.vehicle_type, df.is_automatic)
chi2, p, degf, expected = stats.chi2_contingency(obv)
p

In [ ]:
# Is an employee's gender independent of whether an employee works in sales or marketing? 

In [ ]:
sql4 = """
SELECT gender, dept_name
FROM employees
JOIN dept_emp using(emp_no)
JOIN departments using(dept_no)
WHERE to_date > curdate()
AND dept_name IN ('Sales', 'Marketing')
"""

url2 = get_db_url("employees")

df = pd.read_sql(sql4, url2)

In [ ]:
obv = pd.crosstab(df.gender, df.dept_name)
obv

In [ ]:
# Is an employee's gender independent of whether or not they are or have been a manager?

In [ ]:
sql5 = """
SELECT gender, title, emp_no
FROM employees
JOIN titles using(emp_no)
"""
df = pd.read_sql(sql5, url2)
df["is_manager"] = df.title == "Manager"

In [ ]:
obv = pd.crosstab(df.gender, df.is_manager)
obv

In [ ]:
chi2, p, degf, expected = stats.chi2_contingency(observed)
p